In [1]:
# Make a copick project
import os
import shutil

config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
          "name" : "beta-amylase",
            "is_particle": true,
            "pdb_id": "8ZRZ",
            "label": 2,
            "color": [255, 255, 255, 128],
            "radius": 90,
            "map_threshold": 0.0578  
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        },
        {
            "name": "membrane",
            "is_particle": false,
            "label": 8,
            "color": [100, 100, 100, 128]
        },
        {
            "name": "background",
            "is_particle": false,
            "label": 9,
            "color": [10, 150, 200, 128]
        }
    ],

    "overlay_root": "./kaggle/working/overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "./kaggle/input/czii-cryo-et-object-identification/train/static"
}"""

copick_config_path = "./kaggle/working/copick.config"
output_overlay = "./kaggle/working/overlay"


with open(copick_config_path, "w") as f:
    f.write(config_blob)
    
# Update the overlay
# Define source and destination directories
source_dir = './kaggle/input/czii-cryo-et-object-identification/train/overlay'
destination_dir = './kaggle/working/overlay'

# Walk through the source directory
for root, dirs, files in os.walk(source_dir):
    # Create corresponding subdirectories in the destination
    relative_path = os.path.relpath(root, source_dir)
    target_dir = os.path.join(destination_dir, relative_path)
    os.makedirs(target_dir, exist_ok=True)
    
    # Copy and rename each file
    for file in files:
        if file.startswith("curation_0_"):
            new_filename = file
        else:
            new_filename = f"curation_0_{file}"
            
        
        # Define full paths for the source and destination files
        source_file = os.path.join(root, file)
        destination_file = os.path.join(target_dir, new_filename)
        
        # Copy the file with the new name
        shutil.copy2(source_file, destination_file)
        print(f"Copied {source_file} to {destination_file}")

Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/ribosome.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_ribosome.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/virus-like-particle.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_virus-like-particle.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-galactosidase.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_beta-galactosidase.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-amylase.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_beta-amylase.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/c

In [2]:
import copick
import numpy as np
from tqdm import tqdm

root = copick.from_file(copick_config_path)

copick_user_name = "copickUtils"
copick_segmentation_name = "paintedPicks"
voxel_size = 10

/var/folders/l1/vt4hj2ln3hs6c97_qllsy7lh0000gn/T/ipykernel_15094/1521813977.py:5: DeprecationWarning: config_type not found in config file, defaulting to filesystem
  root = copick.from_file(copick_config_path)


In [5]:
import os
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt

# Define tomogram types
tomo_tpye_list = ["ctfdeconvolved", "denoised", "isonetcorrected", "wbp"]

# Configuration for directories
train_label_dir = Path('./datasets/labels/train')
train_image_dir = Path('./datasets/images/train')
val_label_dir = Path('./datasets/labels/val')
val_image_dir = Path('./datasets/images/val')

for dir_path in [train_label_dir, train_image_dir, val_label_dir, val_image_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Iterate over all tomogram types
for tomo_type in tomo_tpye_list:
    print(f"Processing \"{tomo_type}\" data...")
    for vol_idx, run in enumerate(root.runs):
        # Load image and label data
        tomogram = run.get_voxel_spacing(voxel_size).get_tomogram(tomo_type).numpy()
        segmentation = run.get_segmentations(
            name=copick_segmentation_name,
            user_id=copick_user_name,
            voxel_size=voxel_size,
            is_multilabel=True
        )[0].numpy()

        data_dict = {"image": tomogram, "label": segmentation}

        # Determine dataset type (train/val)
        is_test = (vol_idx == len(root.runs) - 1)
        label_dir = val_label_dir if is_test else train_label_dir
        image_dir = val_image_dir if is_test else train_image_dir

        # Save slices for current tomogram type
        for slice_idx in range(data_dict["image"].shape[0]):  # Iterate over slices
            base_filename = f"{tomo_type}_vol_{vol_idx:01d}_slice_{slice_idx:03d}"

            # Save label as PNG
            label_slice = data_dict["label"][slice_idx]
            plt.imsave(label_dir / f"{base_filename}.png", label_slice, cmap='gray')

            # Normalize and save image
            slice_img = data_dict["image"][slice_idx]
            norm_img = ((slice_img - slice_img.min()) / (slice_img.max() - slice_img.min()) * 255).astype(np.uint8)
            plt.imsave(image_dir / f"{base_filename}.png", norm_img, cmap='gray')

        print(f"Processed volume {vol_idx} for \"{tomo_type}\" dataset.")

Processing "ctfdeconvolved" data...
Processed volume 0 for "ctfdeconvolved" dataset.
Processed volume 1 for "ctfdeconvolved" dataset.
Processed volume 2 for "ctfdeconvolved" dataset.
Processed volume 3 for "ctfdeconvolved" dataset.
Processed volume 4 for "ctfdeconvolved" dataset.
Processed volume 5 for "ctfdeconvolved" dataset.
Processed volume 6 for "ctfdeconvolved" dataset.
Processing "denoised" data...
Processed volume 0 for "denoised" dataset.
Processed volume 1 for "denoised" dataset.
Processed volume 2 for "denoised" dataset.
Processed volume 3 for "denoised" dataset.
Processed volume 4 for "denoised" dataset.
Processed volume 5 for "denoised" dataset.
Processed volume 6 for "denoised" dataset.
Processing "isonetcorrected" data...
Processed volume 0 for "isonetcorrected" dataset.
Processed volume 1 for "isonetcorrected" dataset.
Processed volume 2 for "isonetcorrected" dataset.
Processed volume 3 for "isonetcorrected" dataset.
Processed volume 4 for "isonetcorrected" dataset.
Pro

In [1]:
image_dir = Path(image_dir)
label_dir = Path(label_dir) if label_dir else None





# Collect sorted file paths for images
image_files = sorted(list(image_dir.glob("*.png")))
image_files[:194]

NameError: name 'Path' is not defined

In [20]:
import torch
import torch.optim as optim
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from pathlib import Path

from collections import defaultdict
import numpy as np
import torch
from pathlib import Path
from PIL import Image

class MultiChannelCryoETDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, label_dir=None, num_channels=11, slice_size=(224, 224), stride=112, transform=None, default_label=0):
        """
        Args:
            image_dir (str): 이미지 디렉토리 경로.
            label_dir (str): 라벨 디렉토리 경로 (Optional).
            num_channels (int): 사용할 채널(슬라이스) 수.
            slice_size (tuple): 슬라이스 크기 (W, H).
            stride (int): 슬라이싱 시 stride.
            transform (callable): 데이터 변환 함수.
            default_label (int): 라벨이 없을 때의 기본값.
        """
        self.image_dir = Path(image_dir)
        self.label_dir = Path(label_dir) if label_dir else None
        self.num_channels = num_channels
        self.slice_width, self.slice_height = slice_size
        self.stride = stride
        self.transform = transform
        self.default_label = default_label

        # 볼륨별 슬라이스 그룹화
        self.volume_slices = self._group_slices_by_volume()
        self.slices = self._generate_slices()

    def _group_slices_by_volume(self):
        """
        볼륨별로 슬라이스를 정리하여 그룹화합니다.
        """
        volume_groups = defaultdict(list)
        for file_path in sorted(self.image_dir.glob("*.png")):
            parts = file_path.stem.split("_")
            volume_id = parts[2]  # 예: "vol_0"
            slice_id = int(parts[4])  # 예: "slice_001"
            volume_groups[volume_id].append((slice_id, file_path))
        
        for key in volume_groups:
            volume_groups[key] = sorted(volume_groups[key], key=lambda x: x[0])
        
        return volume_groups

    def _generate_slices(self):
        """
        슬라이싱 가능한 모든 범위를 생성합니다.
        """
        slices = []
        for volume_id, slice_list in self.volume_slices.items():
            for i in range(0, len(slice_list) - self.num_channels + 1):
                slice_range = slice_list[i:i + self.num_channels]
                slices.append({"volume_id": volume_id, "slice_range": slice_range})
        return slices

    def __len__(self):
        return len(self.slices)

    def __getitem__(self, idx):
        slice_info = self.slices[idx]
        slice_range = slice_info["slice_range"]
        # volume_id = slice_info["volume_id"]  # 볼륨 ID 추가

        # 다채널 이미지 생성
        channels = []
        for _, file_path in slice_range:
            img = np.array(Image.open(file_path).convert("L"), dtype=np.float32)
            img = self._resize_slice(img)
            channels.append(img)

        input_tensor = torch.tensor(np.stack(channels, axis=0))  # (num_channels, W, H)

        # 라벨 처리
        if self.label_dir:
            label_path = self.label_dir / slice_range[0][1].name
            label = np.array(Image.open(label_path).convert("L"), dtype=np.int64)
            label = self._resize_slice(label)
            label_tensor = torch.tensor(label, dtype=torch.long)
        else:
            label_tensor = torch.full((self.slice_width, self.slice_height), self.default_label, dtype=torch.long)

        # Transform 적용
        if self.transform:
            input_tensor, label_tensor = self.transform(input_tensor, label_tensor)

        # 볼륨 ID와 함께 반환
        return input_tensor, label_tensor, 

    def _resize_slice(self, slice_img):
        """
        슬라이스 크기를 `slice_size`에 맞게 리샘플링합니다.
        """
        # Ensure slice_img is in a compatible format (uint8 or float32)
        if slice_img.dtype not in [np.uint8, np.float32]:
            slice_img = slice_img.astype(np.float32)
        
        slice_img = Image.fromarray(slice_img)
        slice_img = slice_img.resize((self.slice_width, self.slice_height), Image.BILINEAR)
        return np.array(slice_img, dtype=np.float32)
    
# 데이터셋 경로 설정
image_dir = "./datasets/images/train"
label_dir = "./datasets/labels/train"

# Dataset과 DataLoader 초기화
dataset = MultiChannelCryoETDataset(image_dir=image_dir, label_dir=label_dir, num_channels=11)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 데이터 확인
for inputs, labels in dataloader:
    
    print(f"Input shape: {inputs.shape}, Label shape: {labels.shape}")
    break

Input shape: torch.Size([2, 11, 224, 224]), Label shape: torch.Size([2, 224, 224])


In [ ]:
# 모델 정의
model = UNet(
    spatial_dims=2,
    in_channels=11,  # 입력 채널
    out_channels=6,  # 클래스 수
    channels=(64, 128, 256, 512, 1024),
    strides=(2, 2, 2, 2)
)

# 데이터 확인 및 학습 루프
for inputs, labels in dataloader:
    outputs = model(inputs)  # Forward pass
    print(f"Model output shape: {outputs.shape}")
    break

In [ ]:
# Loss and Optimizer
criterion = DiceLoss(to_onehot_y=True, softmax=True)  # Dice Loss with softmax
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets.unsqueeze(1))  # Compute Dice Loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
# import numpy as np

# # Define the model (U-Net)
# class DoubleConv(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(DoubleConv, self).__init__()
#         self.double_conv = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x):
#         return self.double_conv(x)


# class UNet(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(UNet, self).__init__()
#         self.enc1 = DoubleConv(in_channels, 64)
#         self.pool1 = nn.MaxPool2d(2)
#         self.enc2 = DoubleConv(64, 128)
#         self.pool2 = nn.MaxPool2d(2)
#         self.enc3 = DoubleConv(128, 256)
#         self.pool3 = nn.MaxPool2d(2)
#         self.bridge = DoubleConv(256, 512)
#         self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
#         self.dec3 = DoubleConv(512, 256)
#         self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
#         self.dec2 = DoubleConv(256, 128)
#         self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
#         self.dec1 = DoubleConv(128, 64)
#         self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

#     def forward(self, x):
#         enc1 = self.enc1(x)
#         enc2 = self.enc2(self.pool1(enc1))
#         enc3 = self.enc3(self.pool2(enc2))
#         bridge = self.bridge(self.pool3(enc3))
#         dec3 = self.dec3(torch.cat([self.upconv3(bridge), enc3], dim=1))
#         dec2 = self.dec2(torch.cat([self.upconv2(dec3), enc2], dim=1))
#         dec1 = self.dec1(torch.cat([self.upconv1(dec2), enc1], dim=1))
#         return self.out_conv(dec1)


# # Instantiate model, loss, and optimizer
# model = UNet(in_channels=11, out_channels=2)  # 2 classes for segmentation
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-4)


# # Training Loop
# for epoch in range(5):  # 5 epochs for demonstration
#     model.train()
#     running_loss = 0.0
#     for inputs, targets in dataloader:
#         optimizer.zero_grad()
#         outputs = model(inputs)  # Forward pass
#         loss = criterion(outputs, targets)  # Compute loss
#         loss.backward()  # Backpropagation
#         optimizer.step()  # Update weights
#         running_loss += loss.item()

#     print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(dataloader):.4f}")

Epoch [1/5], Loss: 0.6931
Epoch [2/5], Loss: 0.6931
Epoch [3/5], Loss: 0.6931
Epoch [4/5], Loss: 0.6931
Epoch [5/5], Loss: 0.6931
